El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (⭐⭐)

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285398 sha256=498761b7a8b9c7b443f209781cf01fac691edfb845e26b22819f3331d649482b
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubu

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df_contenido = sqlContext.read.csv('/content/drive/MyDrive/contents.csv', header=True, inferSchema=True, multiLine=True, escape='"')
rdd_contenido = df_contenido.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Primero necesito agrupar los Usuarios que tengan 1, >10 y > 100 comentarios entre todos los contenidos

In [ ]:
rdd_filtrado = rdd_contenido.map(lambda x: (x.revisor_username, x.revisor_ip, x.revisor_comment))

Veo la cantidad de Articulos sin comentarios para los usarios registrados y sin registrar

Para los registrados

In [ ]:
rdd_usuarios_registrados = rdd_filtrado.filter(lambda x: x[0] != None).cache()

In [ ]:
rdd_usuarios_registrados = rdd_usuarios_registrados.map(lambda x: (x[0], x[2]))

Con los usuarios no registrados

In [ ]:
rdd_usuarios_no_registrados = rdd_filtrado.filter(lambda x: x[1] != None).cache()

In [ ]:
rdd_usuarios_no_registrados = rdd_usuarios_no_registrados.map(lambda x: (x[1], x[2]))

Agrupo los usuarios que revisaron un articulo y no comentaron nada




Primero con los usuarios registrados

In [ ]:
rdd_ur_sin_comentarios = rdd_usuarios_registrados.filter(lambda x: x[1] == None).cache()

In [ ]:
rdd_ur_sin_comentarios = rdd_ur_sin_comentarios.map(lambda x: (x[0], 1))

In [ ]:
rdd_ur_sin_comentarios = rdd_ur_sin_comentarios.reduceByKey(lambda x, y: x + y).cache()

Con los no registrados

In [ ]:
rdd_unr_sin_comentarios = rdd_usuarios_no_registrados.filter(lambda x: x[1] == None).cache()

In [ ]:
rdd_unr_sin_comentarios = rdd_unr_sin_comentarios.map(lambda x: (x[0], 1))

In [ ]:
rdd_unr_sin_comentarios = rdd_unr_sin_comentarios.reduceByKey(lambda x, y: x + y).cache()

Caculo las cantidades por los grupos, en 1 es con los usarios con una revision, 10 con los usuarios con mas de 10 revision y 100 con los usarios con mas de 100 revisiones.

Con los usuarios registrados (ur)

In [ ]:
cantidad_ur_sin_comentarios_1 = rdd_ur_sin_comentarios.filter(lambda x: x[1] == 1).map(lambda x: x[1]).sum()
cantidad_ur_sin_comentarios_10 = rdd_ur_sin_comentarios.filter(lambda x: x[1] > 10).map(lambda x: x[1]).sum()
cantidad_ur_sin_comentarios_100 = rdd_ur_sin_comentarios.filter(lambda x: x[1] > 100).map(lambda x: x[1]).sum()

Con los usuarios no registrados (unr)

In [ ]:
cantidad_unr_sin_comentarios_1 = rdd_unr_sin_comentarios.filter(lambda x: x[1] == 1).map(lambda x: x[1]).sum()
cantidad_unr_sin_comentarios_10 = rdd_unr_sin_comentarios.filter(lambda x: x[1] > 10).map(lambda x: x[1]).sum()
cantidad_unr_sin_comentarios_100 = rdd_unr_sin_comentarios.filter(lambda x: x[1] > 100).map(lambda x: x[1]).sum()

El total de articulos sin comentarios por todos los usuarios en los grupos es la suma de los no registrados con los registrados


In [ ]:
 total_articulos_sin_comentarios_1 = cantidad_ur_sin_comentarios_1 + cantidad_unr_sin_comentarios_1
 total_articulos_sin_comentarios_10 = cantidad_ur_sin_comentarios_10 + cantidad_unr_sin_comentarios_10
 total_articulos_sin_comentarios_100 = cantidad_ur_sin_comentarios_100 + cantidad_unr_sin_comentarios_100

Tengo las cantidades de los articulos sin comentarios en cada subgrupo, veo que pasa con los usarios registrados y no resgistrados y veo cuantos comentaron.

In [ ]:
rdd_ur_comentaron = rdd_usuarios_registrados.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)

In [ ]:
rdd_unr_comentaron = rdd_usuarios_no_registrados.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)

Creo todos los sub grupos

In [ ]:
cantidad_comentarios_ur_comentaron_1 = rdd_ur_comentaron.filter(lambda x: x[1] == 1).map(lambda x: x[1]).sum()
cantidad_comentarios_ur_comentaron_10 = rdd_ur_comentaron.filter(lambda x: x[1] > 10).map(lambda x: x[1]).sum()
cantidad_comentarios_ur_comentaron_100 = rdd_ur_comentaron.filter(lambda x: x[1] > 100).map(lambda x: x[1]).sum()

In [ ]:
cantidad_comentarios_unr_comentaron_1 = rdd_unr_comentaron.filter(lambda x: x[1] == 1).map(lambda x: x[1]).sum()
cantidad_comentarios_unr_comentaron_10 = rdd_unr_comentaron.filter(lambda x: x[1] > 10).map(lambda x: x[1]).sum()
cantidad_comentarios_unr_comentaron_100 = rdd_unr_comentaron.filter(lambda x: x[1] > 100).map(lambda x: x[1]).sum()

Veo las cantidad total de comentarios para cada grupo

In [ ]:
cantidad_comentarios_1 = cantidad_comentarios_ur_comentaron_1 + cantidad_comentarios_unr_comentaron_1

In [ ]:
cantidad_comentarios_10 = cantidad_comentarios_ur_comentaron_10 + cantidad_comentarios_unr_comentaron_10

In [ ]:
cantidad_comentarios_100 = cantidad_comentarios_ur_comentaron_100 + cantidad_comentarios_unr_comentaron_100

Calculo los porcentajes pedidos para cada grupo

In [ ]:
porcentaje_1 = 100*(total_articulos_sin_comentarios_1/cantidad_comentarios_1)

In [ ]:
porcentaje_10 = 100*(total_articulos_sin_comentarios_10)/(cantidad_comentarios_10)

In [ ]:
porcentaje_100 = 100*(total_articulos_sin_comentarios_100)/(cantidad_comentarios_100)

In [ ]:
porcentaje_1

40.43636300241455

In [ ]:
porcentaje_10

9.830704298834851

In [ ]:
porcentaje_100

8.417951820682088

En mi caso, yo considere tanto a los usarios registrados como los no registrados ya que no se hace mencion en el enunciado de trabajar con algunos de los dos tipos de usuarios.